# ÖNEMLİ BİLGİLENDİRME

**Bu notebook, modelin eğitimi için Google Colab ortamında çalıştırılmıştır.**

Kod içerisinde `google.colab` kütüphanesi ve `kaggle` komutları gibi sadece Colab ortamına özgü komutlar bulunmaktadır. 

**Eğitilmiş model (`traffic_classifier_v2.h5`), `models/` klasöründe mevcuttur.**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import os
from google.colab import files
from sklearn.model_selection import train_test_split

# VERİ İNDİRME

if not os.path.exists('gtsrb-german-traffic-sign.zip'):
    print("Lütfen 'kaggle.json' dosyasını yükleyin:")
    uploaded = files.upload()
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    print("Veri seti indiriliyor...")
    !kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign --force
    print("Zip dosyası açılıyor...")
    !unzip -q gtsrb-german-traffic-sign.zip

print("Veri seti hazır!")

# VERİ İŞLEME
data = []
labels = []
classes = 43
cur_path = os.getcwd()

print("Görseller okunuyor... (Lütfen bekleyin)")

for i in range(classes):
    path = os.path.join(cur_path, 'Train', str(i))
    if not os.path.exists(path):
        continue
    images = os.listdir(path)
    for a in images:
        try:
            image_path = os.path.join(path, a)
            image = Image.open(image_path)
            image = image.resize((30,30)) # Tüm resimler 30x30
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except Exception as e:
            print(f"Hata: {e}")

# NORMALİZASYON
data = np.array(data) / 255.0  
labels = np.array(labels)

print(f"Veri yüklendi. Boyut: {data.shape}")

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

# DATA AUGMENTATION
aug = ImageDataGenerator(
    rotation_range=10,      # 10 derece döndür
    zoom_range=0.15,        # %15 yakınlaş/uzaklaş
    width_shift_range=0.1,  # Sağa sola kaydır
    height_shift_range=0.1, # Yukarı aşağı kaydır
    shear_range=0.15,       # Resmi biraz eğ
    horizontal_flip=False,  # Trafik levhası ters dönmez 
    fill_mode="nearest"
)

# MODEL MİMARİSİ
model = Sequential()

# Blok 1
model.add(Conv2D(32, (5,5), activation='relu', input_shape=(30,30,3)))
model.add(Conv2D(32, (5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Blok 2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Sınıflandırma
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(43, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# EĞİTİM
print("Model eğitimi başlıyor...")
batch_size = 32
epochs = 15

history = model.fit(aug.flow(X_train, y_train, batch_size=batch_size),
                    epochs=epochs,
                    validation_data=(X_test, y_test))

# GRAFİKLER 
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Eğitim')
plt.plot(history.history['val_accuracy'], label='Test')
plt.title('Doğruluk (Accuracy)')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Eğitim')
plt.plot(history.history['val_loss'], label='Test')
plt.title('Kayıp (Loss)')
plt.legend()
plt.show()

model.save("traffic_classifier_v2.h5")
print(" Model kaydedildi: traffic_classifier_v2.h5")